# Final Runs for the MLP model

In [ ]:
import os
import json
import numpy as np
from structure import run_experiments, make_optimizer_config

# === Step 1 : grid search ===
optim_names = ["PGD", "SGD", "Nesterov","GD", "PartialGD", "Momentum", "RMSprop", "Adam"]
lr_grid = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
dataset_list = ["SVHN", "CIFAR10"]
model_list = [{"name": "MLP"}]

optim_configs = []
for name in optim_names:
    for lr in lr_grid:
        if name in ["Momentum", "Nesterov"]:
            optim_configs.append(make_optimizer_config(name, lr=lr, momentum=0.9))
        elif name == "PartialGD":
            optim_configs.append(make_optimizer_config(name, lr=lr, update_fraction=0.5))
        elif name == "PGD":
            optim_configs.append(make_optimizer_config(name, lr=lr))
        else:
            optim_configs.append(make_optimizer_config(name, lr=lr))
# !export USE_GPU=true
# === Run the grid scan ===
results_path = "data/grid_scan_MLP.json"
run_experiments(
    datasets=dataset_list,
    models=model_list,
    optimizers_with_params=optim_configs,
    epochs=30,
    save_results=True,
    save_path=results_path
)


In [ ]:

filename = "data/grid_scan_MLP.json"

# === Step 2 : extract the best lr and epochs ===
with open(filename, "r") as f:
    all_results = json.load(f)

best_configs = {}
for r in all_results:
    key = (r["dataset"], r["optimizer"])
    test_losses = r.get("test_losses", [])
    if not test_losses:
        continue
    min_epoch = int(np.argmin(test_losses))
    min_loss = test_losses[min_epoch]
    if key not in best_configs or min_loss < min(best_configs[key]["test_losses"]):
        best_configs[key] = {
            "dataset": r["dataset"],
            "optimizer": r["optimizer"],
            "params": r["optimizer_params"],
            "best_epoch": min_epoch,
            "min_loss": min_loss,
            "test_losses": test_losses  # ✅ Ajout nécessaire

        }

# === Step 3: rerun the best runs with optimal lr and adjusted epoch ===
for (dataset, optim), config in best_configs.items():
    lr = config["params"]["lr"]
    best_epoch = config["best_epoch"]
    adjusted_epoch = best_epoch 

    print(f"\n🔁 Running {optim} on {dataset} with lr={lr:.5f} for {adjusted_epoch} epochs")

    run_experiments(
        datasets=[dataset],
        models=model_list,
        optimizers_with_params=[{
            "name": optim,
            "params": config["params"]
        }],
        epochs=adjusted_epoch,
        avg_over_seeds=True,
        seeds=[0, 1, 2, 3, 4],
        save_results=True,
        save_path=f"Data/MLP_best_{dataset}_{optim}.json"
    )


# Merge the files

In [ ]:
import os
import json

def merge_mlp_best_results(data_dir="data"):
    """
    Merges all files MLP_best_{dataset}_{optimizer}.json into:
    - data/merged_MLP_CIFAR10.json
    - data/merged_MLP_SVHN.json
    """
    merged = {"CIFAR10": [], "SVHN": []}

    for filename in os.listdir(data_dir):
        if filename.startswith("MLP_best_") and filename.endswith(".json"):
            if "CIFAR10" in filename:
                dataset = "CIFAR10"
            elif "SVHN" in filename:
                dataset = "SVHN"
            else:
                continue  # ignore the datasets

            filepath = os.path.join(data_dir, filename)
            with open(filepath, "r") as f:
                data = json.load(f)
                if isinstance(data, list):
                    merged[dataset].extend(data)
                else:
                    merged[dataset].append(data)

    # Final save
    for dataset in merged:
        output_file = os.path.join(data_dir, f"merged_MLP_{dataset}.json")
        with open(output_file, "w") as f_out:
            json.dump(merged[dataset], f_out, indent=2)
        print(f"✅ Fusion terminée pour {dataset}: {output_file}")

# Direct execution
merge_mlp_best_results()
